 # 웹크롤링과 텍스트데이터 시각화

### Outline

### 7. 텍스트데이터 시각화 (A. 워드클라우드)
    7A.1 텍스트데이터 수집 및 전처리
    7A.2 워드클라우드 생성

#### 7A.1 텍스트데이터 수집 및 전처리

- 콘솔창에서 wordcloud 라이브러리 설치하기

In [ ]:
!pip install wordcloud

- 필요한 라이브러리, 패키지, 모듈 불러오기

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as MImage
from IPython.display import Image
import re
import requests as req
from bs4 import BeautifulSoup as bs
from wordcloud import WordCloud, STOPWORDS

- 형태소 분석 라이브러리, nltk (natural language toolkit)

In [ ]:
# 명사 추출
import nltk
nltk.download('averaged_perceptron_tagger')

# 영어 불용어(stopwords) 사전
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

# 단어/문장 단위로 토큰화
nltk.download('punkt')
from nltk.tokenize import word_tokenize,sent_tokenize

- 타겟 URL에서 기사 내용 가져오기

In [ ]:
Image(filename='Article.png', width=700)

In [ ]:
url  = 'https://www.google.com/amp/s/nymag.com/intelligencer/amp/2020/05/scott-galloway-future-of-college.html'
res  = req.get(url)
soup = bs(res.text,'html.parser')

In [ ]:
Image(filename='Selector.png', width=700)

In [ ]:
text = soup.select('section > div.article-content.special-feature')[0].text

In [ ]:
text[463:637]

(1) 공백 제거

In [ ]:
text_filtered = text.replace('\n','').replace('\u200b','')

(2) 불필요 기호 제거 + 소문자화

In [ ]:
text_filtered = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\-\:\;\!\-\,\_\~\$\'\"]',' ',text_filtered.lower())

(3) 숫자 제거

In [ ]:
text_filtered = re.sub('\d',' ',text_filtered)

(4) 토큰화 하기

- 불용어를 제거해주기 위해서, 단어 단위로 "토큰화" 실행

In [ ]:
word_tokens = word_tokenize(text_filtered)

(5) 명사 추출

- 각 단어들의 품사를 할당

In [ ]:
word_tokens_tag = nltk.pos_tag(word_tokens)
word_tokens_tag[:10]

- wordcloud로 텍스트데이터를 시각화하기 위해, 명사만을 추출

In [ ]:
word_tokens = [word for (word, pos) in word_tokens_tag if (pos[:2] == 'NN')]

(6) 3글자 이상의 단어만 추출

In [ ]:
word_tokens = [word for word in word_tokens if len(word)>2]

(7) 기본적인 불용어 처리

- nltk 패키지에 등재된 자주 사용되는 불용어

In [ ]:
list(stopwords)[:10] # 집합 자료형으로 리스트로 변환해 슬라이싱

In [ ]:
result = [w for w in word_tokens if w not in stopwords]

(8) 불용어 추가 및 단어 수정

- 최빈단어 확인 후, 추가할 불용어가 있으면 수작업으로 추가

In [ ]:
from collections import Counter
word_freq = Counter(result)
word_freq.most_common()

In [ ]:
stopwords2 = ['year','part']
result = [w for w in result if w not in stopwords2]

- 맞춤법이 틀리거나 수정이 필요한 단어를 수정

In [ ]:
result = [re.sub("universities","university",x) for x in result]

In [ ]:
word_freq = Counter(result)
word_freq.most_common()

(9) 토큰화된 단어를 다시 이어서 리스트로 변환

In [ ]:
words = ' '.join(result)

#### 7A.2 워드클라우드 생성

- 워드클라우드의 모양 설정

In [ ]:
mask = np.array(MImage.open("University.png"))
mask = mask[:,:,0]

In [ ]:
Image(filename='University.png', width=300)

- 워드클라우드 생성

In [ ]:
wc = WordCloud(background_color="white",mask=mask)
wc = wc.generate(words)

In [ ]:
wc.words_

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wc.to_file('wordcloud.jpg')